In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, make_scorer, mean_absolute_percentage_error
from scipy.stats import loguniform
import joblib
import tensorflow as tf
from sklearn.model_selection import ParameterSampler

# Expand display options
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.max_colwidth", None)  # Do not truncate column text
pd.set_option("display.expand_frame_repr", False)  # Avoid line wrapping

tf.config.threading.set_intra_op_parallelism_threads(18)
tf.config.threading.set_inter_op_parallelism_threads(18)

import optuna
from optuna.integration import TFKerasPruningCallback

In [2]:
df_OHE = pd.read_csv('../DataSet/RegressionData/healthinsurance_OHE.csv')
print(df_OHE.shape)

(13648, 146)


In [3]:
X = df_OHE.drop(columns='claim')
y = df_OHE['claim']

# First split: 70% Training, 30% Temporary (Validation + Test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=69)

# Second split: Splitting the 30% temporary set into 15% Validation and 15% Test
X_eval, X_test, y_eval, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=69)

# Standardization
scaler = StandardScaler()
scaler.fit(X_train)

# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

# Transform the datasets
X_train_scaled = scaler.transform(X_train)
X_eval_scaled = scaler.transform(X_eval)
X_test_scaled = scaler.transform(X_test)

#print dist.
print('Training set: ', X_train_scaled.shape)
print('Validation set: ', X_eval_scaled.shape)
print('Test set: ', X_test_scaled.shape)

Training set:  (9553, 145)
Validation set:  (2047, 145)
Test set:  (2048, 145)


In [12]:
import tensorflow as tf
import optuna
from optuna.integration import TFKerasPruningCallback

# Assume these are defined elsewhere:
# X, y, X_train_scaled, y_train, X_eval_scaled, y_eval, X_test, y_test
input_dim = X.shape[1]

def preprocess(x, y):
    """Optional preprocessing before batching."""
    return x, y

def create_tf_dataset(X, y, batch_size=64, shuffle=True, cache_data=True):
    """Creates a tf.data.Dataset with shuffling, optional caching, batching, and prefetching."""
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(X), reshuffle_each_iteration=True)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if cache_data:
        dataset = dataset.cache()  # Cache data in memory to speed up later epochs
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def create_model(hidden_layers, neurons, optimizer, learning_rate, regularization, dropout_rate):
    """Builds and compiles a Keras model using an explicit Input layer and best practices."""
    tf.keras.backend.clear_session()
    
    model = tf.keras.Sequential([tf.keras.Input(shape=(input_dim,))])
    
    # First Dense layer with batch normalization, activation, and dropout.
    model.add(tf.keras.layers.Dense(
        neurons,
        kernel_initializer=tf.keras.initializers.HeNormal(),
        kernel_regularizer=tf.keras.regularizers.l2(regularization)
    ))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(dropout_rate))
    
    # Additional hidden layers.
    for _ in range(hidden_layers - 1):
        model.add(tf.keras.layers.Dense(
            neurons,
            kernel_initializer=tf.keras.initializers.HeNormal(),
            kernel_regularizer=tf.keras.regularizers.l2(regularization)
        ))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dropout(dropout_rate))
        
    # Output layer.
    model.add(tf.keras.layers.Dense(
        1,
        activation='linear',
        kernel_initializer=tf.keras.initializers.HeNormal(),
        kernel_regularizer=tf.keras.regularizers.l2(regularization)
    ))
    
    # Select and configure the optimizer.
    optimizers = {
        'adamW': tf.keras.optimizers.AdamW(learning_rate=learning_rate),
        'Adam': tf.keras.optimizers.Adam(learning_rate=learning_rate),
        'Nadam': tf.keras.optimizers.Nadam(learning_rate=learning_rate)
    }
    opt = optimizers.get(optimizer, None)
    if opt is None:
        raise ValueError(f"Unsupported optimizer: {optimizer}")

    model.compile(optimizer=opt,
                  loss='mse',
                  metrics=[
                      'mae',
                      tf.keras.metrics.RootMeanSquaredError(),
                      tf.keras.metrics.MeanAbsolutePercentageError()
                  ])
    return model

# Create datasets (only once, outside objective).
train_dataset = create_tf_dataset(X_train_scaled, y_train, batch_size=64, shuffle=True, cache_data=True)
eval_dataset  = create_tf_dataset(X_eval_scaled, y_eval, batch_size=64, shuffle=False, cache_data=True)
test_dataset  = create_tf_dataset(X_test, y_test, batch_size=64, shuffle=False, cache_data=True)

def objective(trial):
    """Objective function for Optuna hyperparameter tuning."""
    # ---------------------
    # 1) Define hyperparameter search space
    # ---------------------
    hidden_layers = trial.suggest_int("hidden_layers", 5, 30)
    neurons = trial.suggest_categorical("neurons", [20, 40, 60, 80, 100])
    optimizer_choice = trial.suggest_categorical("optimizer", ["adamW", "Nadam", "Adam"])
    learning_rate = trial.suggest_float("learning_rate", 0.0005, 0.005, log=True)
    regularization = trial.suggest_categorical("regularization", [0.0001, 0.001, 0.01])
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.3)
    
    # ---------------------
    # 2) Build the model
    # ---------------------
    model = create_model(hidden_layers, neurons, optimizer_choice, learning_rate, regularization, dropout_rate)
    
    # ---------------------
    # 3) Define callbacks (each trial gets its own set)
    # ---------------------
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_mean_absolute_percentage_error',
        patience=20,
        restore_best_weights=True,
        verbose=0
    )
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_mean_absolute_percentage_error',
        factor=0.5,
        patience=10,
        verbose=0
    )
    # Only the pruning callback is used for trial-level evaluation.
    pruning_callback = TFKerasPruningCallback(trial, monitor='val_mean_absolute_percentage_error')
    
    # ---------------------
    # 4) Train the model
    # ---------------------
    model.fit(
        train_dataset,
        validation_data=eval_dataset,
        epochs=200,
        verbose=1,  
        callbacks=[early_stop, lr_scheduler, pruning_callback]
    )
    
    # ---------------------
    # 5) Evaluate the model
    # ---------------------
    eval_score = model.evaluate(eval_dataset, verbose=0)
    eval_mape = eval_score[3]  # [loss, mae, rmse, mape]
    
    return eval_mape

# ---------------------
# Create & run the Optuna study
# ---------------------
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# ---------------------
# Retrieve the best 10 hyperparameter sets and the best trial overall
# ---------------------
top_trials = sorted(study.trials, key=lambda t: t.value)[:10]
best_params_list = [trial.params for trial in top_trials]

print("\nTop 10 Hyperparameter Sets:")
for i, params in enumerate(best_params_list, 1):
    print(f"Rank {i}: {params}")

best_trial = study.best_trial
best_params = best_trial.params

print("\nBest Hyperparameters:")
print(best_params)
print(f"Best Eval MAPE: {best_trial.value:.2f}%")

# ---------------------
# Build best model using best parameters
# ---------------------
best_model = create_model(**best_params)

# ---------------------
# Define final callbacks for training the best model
# ---------------------
final_early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_mean_absolute_percentage_error',
    patience=10,
    restore_best_weights=True,
    verbose=1
)
final_lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_mean_absolute_percentage_error',
    factor=0.5,
    patience=5,
    verbose=1
)
# Save only the best model during final training.
final_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model_final.keras',
    monitor='val_mean_absolute_percentage_error',
    save_best_only=True,
    verbose=1
)

# ---------------------
# Train best model fully (only the best final model is saved)
# ---------------------
best_model.fit(
    train_dataset,
    validation_data=eval_dataset,
    epochs=200,
    verbose=0,
    callbacks=[final_early_stop, final_lr_scheduler, final_checkpoint]
)

# ---------------------
# Evaluate final model
# ---------------------
train_score = best_model.evaluate(train_dataset, verbose=0)
eval_score  = best_model.evaluate(eval_dataset, verbose=0)
test_score  = best_model.evaluate(test_dataset, verbose=0)

print("\n--- Performance of the Best Model ---")
print(f"Train  -> Loss: {train_score[0]:.4f}, MAE: {train_score[1]:.4f}, "
      f"RMSE: {train_score[2]:.4f}, MAPE: {train_score[3]:.2f}%")
print(f"Eval   -> Loss: {eval_score[0]:.4f}, MAE: {eval_score[1]:.4f}, "
      f"RMSE: {eval_score[2]:.4f}, MAPE: {eval_score[3]:.2f}%")
print(f"Test   -> Loss: {test_score[0]:.4f}, MAE: {test_score[1]:.4f}, "
      f"RMSE: {test_score[2]:.4f}, MAPE: {test_score[3]:.2f}%")



[I 2025-02-17 21:06:30,480] A new study created in memory with name: no-name-62a5b0e6-28b8-4932-89f7-6d4bc21b55b8


Epoch 1/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 88s 64ms/step - loss: 321381984.0000 - mae: 13319.4541 - mean_absolute_percentage_error: 99.9977 - root_mean_squared_error: 17924.4766 - val_loss: 326992672.0000 - val_mae: 13445.4932 - val_mean_absolute_percentage_error: 99.9946 - val_root_mean_squared_error: 18082.9395 - learning_rate: 6.2353e-04
Epoch 2/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 321334528.0000 - mae: 13317.7002 - mean_absolute_percentage_error: 99.9688 - root_mean_squared_error: 17923.1523 - val_loss: 326977472.0000 - val_mae: 13444.9521 - val_mean_absolute_percentage_error: 99.9859 - val_root_mean_squared_error: 18082.5176 - learning_rate: 6.2353e-04
Epoch 3/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 321272512.0000 - mae: 13315.3496 - mean_absolute_percentage_error: 99.9297 - root_mean_squared_error: 17921.4238 - val_loss: 326975264.0000 - val_mae: 13444.8945 - val_mean_absolute_percentage_error: 99.9852 - val_root_mean_squared_error: 18082.4570 - learn

[I 2025-02-17 21:20:26,220] Trial 0 finished with value: 26.995189666748047 and parameters: {'hidden_layers': 25, 'neurons': 40, 'optimizer': 'Adam', 'learning_rate': 0.0006235320682528795, 'regularization': 0.0001, 'dropout_rate': 0.24200452741377398}. Best is trial 0 with value: 26.995189666748047.


Epoch 1/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - loss: 321357632.0000 - mae: 13319.3496 - mean_absolute_percentage_error: 100.0039 - root_mean_squared_error: 17923.8008 - val_loss: 325098432.0000 - val_mae: 13420.0264 - val_mean_absolute_percentage_error: 99.9821 - val_root_mean_squared_error: 18030.4863 - learning_rate: 0.0011
Epoch 2/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 321070080.0000 - mae: 13315.7168 - mean_absolute_percentage_error: 100.0138 - root_mean_squared_error: 17915.7812 - val_loss: 325385600.0000 - val_mae: 13426.4678 - val_mean_absolute_percentage_error: 100.0534 - val_root_mean_squared_error: 18038.4473 - learning_rate: 0.0011
Epoch 3/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 320666880.0000 - mae: 13309.0537 - mean_absolute_percentage_error: 99.9927 - root_mean_squared_error: 17904.5293 - val_loss: 325144768.0000 - val_mae: 13421.6943 - val_mean_absolute_percentage_error: 100.0253 - val_root_mean_squared_error: 18031.7715 - learning_ra

[I 2025-02-17 21:23:37,940] Trial 1 finished with value: 22.102413177490234 and parameters: {'hidden_layers': 12, 'neurons': 40, 'optimizer': 'adamW', 'learning_rate': 0.0011462773920085047, 'regularization': 0.01, 'dropout_rate': 0.11283462245892283}. Best is trial 1 with value: 22.102413177490234.


...

Epoch 1/200
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 321110464.0000 - mae: 13315.6270 - mean_absolute_percentage_error: 100.0014 - root_mean_squared_error: 17916.9102

[I 2025-02-18 00:13:12,196] Trial 48 pruned. Trial was pruned at epoch 0.


Epoch 1/200
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 321139168.0000 - mae: 13315.4863 - mean_absolute_percentage_error: 99.9956 - root_mean_squared_error: 17917.6895

[I 2025-02-18 00:14:07,837] Trial 49 pruned. Trial was pruned at epoch 0.



Top 10 Hyperparameter Sets:
Rank 1: {'hidden_layers': 15, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.0027447053581130367, 'regularization': 0.001, 'dropout_rate': 0.10666397301582889}
Rank 2: {'hidden_layers': 12, 'neurons': 40, 'optimizer': 'adamW', 'learning_rate': 0.0011462773920085047, 'regularization': 0.01, 'dropout_rate': 0.11283462245892283}
Rank 3: {'hidden_layers': 14, 'neurons': 100, 'optimizer': 'Nadam', 'learning_rate': 0.0034392907953942997, 'regularization': 0.001, 'dropout_rate': 0.14696151778307404}
Rank 4: {'hidden_layers': 13, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.003607639946220003, 'regularization': 0.001, 'dropout_rate': 0.1182706740916589}
Rank 5: {'hidden_layers': 13, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.0034114118822873626, 'regularization': 0.001, 'dropout_rate': 0.13685195466975933}
Rank 6: {'hidden_layers': 12, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.004580839776429419, 'regularization': 0.00

In [13]:

print("\nTop 10 Hyperparameter Sets:")
for i, params in enumerate(best_params_list, 1):
    print(f"Rank {i}: {params}")


Top 10 Hyperparameter Sets:
Rank 1: {'hidden_layers': 15, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.0027447053581130367, 'regularization': 0.001, 'dropout_rate': 0.10666397301582889}
Rank 2: {'hidden_layers': 12, 'neurons': 40, 'optimizer': 'adamW', 'learning_rate': 0.0011462773920085047, 'regularization': 0.01, 'dropout_rate': 0.11283462245892283}
Rank 3: {'hidden_layers': 14, 'neurons': 100, 'optimizer': 'Nadam', 'learning_rate': 0.0034392907953942997, 'regularization': 0.001, 'dropout_rate': 0.14696151778307404}
Rank 4: {'hidden_layers': 13, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.003607639946220003, 'regularization': 0.001, 'dropout_rate': 0.1182706740916589}
Rank 5: {'hidden_layers': 13, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.0034114118822873626, 'regularization': 0.001, 'dropout_rate': 0.13685195466975933}
Rank 6: {'hidden_layers': 12, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.004580839776429419, 'regularization': 0.00

In [ ]:
import tensorflow as tf
import numpy as np

# ------------------------------------------------------------------
# Assume these are defined externally:
# X, y: the full dataset (used only for inferring input_dim)
# X_train_scaled, y_train: training data (70%)
# X_eval_scaled, y_eval: validation data (15%)
# X_test_scaled, y_test: test data (15%)
# ------------------------------------------------------------------

# Derive input dimension from X
input_dim = X.shape[1]

def preprocess(x, y):
    """Optional preprocessing before batching."""
    return x, y

def create_tf_dataset(X, y, batch_size=64, shuffle=True, cache_data=True):
    """Creates a tf.data.Dataset with shuffling, optional caching, batching, and prefetching."""
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(X), reshuffle_each_iteration=True)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if cache_data:
        dataset = dataset.cache()
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def create_model(hidden_layers, neurons, optimizer, learning_rate, regularization, dropout_rate):
    """Builds and compiles a Keras model using an explicit Input layer and best practices."""
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential([tf.keras.Input(shape=(input_dim,))])
    
    # Add hidden layers: each layer = Dense -> BatchNorm -> LeakyReLU -> Dropout.
    for _ in range(hidden_layers):
        model.add(tf.keras.layers.Dense(
            neurons,
            kernel_initializer=tf.keras.initializers.HeNormal(),
            kernel_regularizer=tf.keras.regularizers.l2(regularization)
        ))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dropout(dropout_rate))
    
    # Output layer for regression.
    model.add(tf.keras.layers.Dense(
        1,
        activation='linear',
        kernel_initializer=tf.keras.initializers.HeNormal(),
        kernel_regularizer=tf.keras.regularizers.l2(regularization)
    ))
    
    # Select optimizer.
    optimizers = {
        'adamW': tf.keras.optimizers.AdamW(learning_rate=learning_rate),
        'Adam': tf.keras.optimizers.Adam(learning_rate=learning_rate),
        'Nadam': tf.keras.optimizers.Nadam(learning_rate=learning_rate)
    }
    opt = optimizers.get(optimizer, None)
    if opt is None:
        raise ValueError(f"Unsupported optimizer: {optimizer}")
    
    model.compile(optimizer=opt,
                  loss='mse',
                  metrics=[
                      'mae',
                      tf.keras.metrics.RootMeanSquaredError(),
                      tf.keras.metrics.MeanAbsolutePercentageError()
                  ])
    return model

# Create datasets from your predefined splits.
train_dataset = create_tf_dataset(X_train_scaled, y_train, batch_size=64, shuffle=True, cache_data=True)
eval_dataset  = create_tf_dataset(X_eval_scaled, y_eval, batch_size=64, shuffle=False, cache_data=True)
test_dataset  = create_tf_dataset(X_test_scaled, y_test, batch_size=64, shuffle=False, cache_data=True)

# ------------------------------------------------------------------
# Define the top 10 parameter combinations (exactly as provided):
# ------------------------------------------------------------------
top_params_list = best_params_list

# ------------------------------------------------------------------
# Final Training for Each of the Top 10 Combinations.
# For each combination, train a model on the full training set (train_dataset)
# with early stopping (patience = 15 epochs) using the validation set (eval_dataset).
# Then evaluate on train, eval, and test sets.
# ------------------------------------------------------------------
for i, params in enumerate(top_params_list):
    print(f"--- Final Training for Combination {i} ---")
    print("Params:", params)
    
    model = create_model(**params)
    
    final_early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_mean_absolute_percentage_error',
        patience=15,  # Stop if no improvement for 15 epochs
        restore_best_weights=True,
        verbose=1
    )
    final_lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_mean_absolute_percentage_error',
        factor=0.5,
        patience=10,
        verbose=1
    )
    final_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'final_model_combo_{i}.keras',
        monitor='val_mean_absolute_percentage_error',
        save_best_only=True,
        verbose=1
    )
    
    model.fit(
        train_dataset,
        validation_data=eval_dataset,
        epochs=200,
        verbose=1,
        callbacks=[final_early_stop, final_lr_scheduler, final_checkpoint]
    )
    
    train_score = model.evaluate(train_dataset, verbose=0)
    eval_score  = model.evaluate(eval_dataset, verbose=0)
    test_score  = model.evaluate(test_dataset, verbose=0)
    
    print(f"\n--- Performance for Combination {i} ---")
    print(f"Train  -> Loss: {train_score[0]:.4f}, MAE: {train_score[1]:.4f}, RMSE: {train_score[2]:.4f}, MAPE: {train_score[3]:.2f}%")
    print(f"Eval   -> Loss: {eval_score[0]:.4f}, MAE: {eval_score[1]:.4f}, RMSE: {eval_score[2]:.4f}, MAPE: {eval_score[3]:.2f}%")
    print(f"Test   -> Loss: {test_score[0]:.4f}, MAE: {test_score[1]:.4f}, RMSE: {test_score[2]:.4f}, MAPE: {test_score[3]:.2f}%\n")


--- Final Training for Combination 0 ---
Params: {'hidden_layers': 15, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.0027447053581130367, 'regularization': 0.001, 'dropout_rate': 0.10666397301582889}
Epoch 1/200
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 323167200.0000 - mae: 13334.0303 - mean_absolute_percentage_error: 100.0020 - root_mean_squared_error: 17974.9727
Epoch 1: val_mean_absolute_percentage_error improved from inf to 98.33588, saving model to final_model_combo_0.keras
150/150 ━━━━━━━━━━━━━━━━━━━━ 52s 55ms/step - loss: 323184640.0000 - mae: 13334.9883 - mean_absolute_percentage_error: 100.0019 - root_mean_squared_error: 17975.4805 - val_loss: 268664416.0000 - val_mae: 12581.4609 - val_mean_absolute_percentage_error: 98.3359 - val_root_mean_squared_error: 16390.9863 - learning_rate: 0.0027
Epoch 2/200
149/150 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 320570112.0000 - mae: 13291.0068 - mean_absolute_percentage_error: 99.8550 - root_mean_squared_error: 17902.6

In [4]:
import tensorflow as tf
import numpy as np

# ------------------------------------------------------------------
# Assume these are defined externally:
# X, y: the full dataset (used only for inferring input_dim)
# X_train_scaled, y_train: training data (70%)
# X_eval_scaled, y_eval: validation data (15%)
# X_test_scaled, y_test: test data (15%)
# ------------------------------------------------------------------

# Derive input dimension from X
input_dim = X.shape[1]

def preprocess(x, y):
    """Optional preprocessing before batching."""
    return x, y

def create_tf_dataset(X, y, batch_size=64, shuffle=True, cache_data=True):
    """Creates a tf.data.Dataset with shuffling, optional caching, batching, and prefetching."""
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(X), reshuffle_each_iteration=True)
    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    if cache_data:
        dataset = dataset.cache()
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def create_model(hidden_layers, neurons, optimizer, learning_rate, regularization, dropout_rate):
    """Builds and compiles a Keras model using an explicit Input layer and best practices."""
    tf.keras.backend.clear_session()
    model = tf.keras.Sequential([tf.keras.Input(shape=(input_dim,))])
    
    # Add hidden layers: each layer = Dense -> BatchNorm -> LeakyReLU -> Dropout.
    for _ in range(hidden_layers):
        model.add(tf.keras.layers.Dense(
            neurons,
            kernel_initializer=tf.keras.initializers.HeNormal(),
            kernel_regularizer=tf.keras.regularizers.l2(regularization)
        ))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dropout(dropout_rate))
    
    # Output layer for regression.
    model.add(tf.keras.layers.Dense(
        1,
        activation='linear',
        kernel_initializer=tf.keras.initializers.HeNormal(),
        kernel_regularizer=tf.keras.regularizers.l2(regularization)
    ))
    
    # Select optimizer.
    optimizers = {
        'adamW': tf.keras.optimizers.AdamW(learning_rate=learning_rate),
        'Adam': tf.keras.optimizers.Adam(learning_rate=learning_rate),
        'Nadam': tf.keras.optimizers.Nadam(learning_rate=learning_rate)
    }
    opt = optimizers.get(optimizer, None)
    if opt is None:
        raise ValueError(f"Unsupported optimizer: {optimizer}")
    
    model.compile(optimizer=opt,
                  loss='mse',
                  metrics=[
                      'mae',
                      tf.keras.metrics.RootMeanSquaredError(),
                      tf.keras.metrics.MeanAbsolutePercentageError()
                  ])
    return model

# Create datasets from your predefined splits.
train_dataset = create_tf_dataset(X_train_scaled, y_train, batch_size=64, shuffle=True, cache_data=True)
eval_dataset  = create_tf_dataset(X_eval_scaled, y_eval, batch_size=64, shuffle=False, cache_data=True)
test_dataset  = create_tf_dataset(X_test_scaled, y_test, batch_size=64, shuffle=False, cache_data=True)

# ------------------------------------------------------------------
# Define the top 10 parameter combinations (exactly as provided):
# ------------------------------------------------------------------
top_params_list = [
    {'hidden_layers': 15, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.0027447053581130367, 'regularization': 0.001, 'dropout_rate': 0.10666397301582889}
]

# ------------------------------------------------------------------
# Final Training for Each of the Top 10 Combinations.
# For each combination, train a model on the full training set (train_dataset)
# with early stopping (patience = 15 epochs) using the validation set (eval_dataset).
# Then evaluate on train, eval, and test sets.
# ------------------------------------------------------------------
for i, params in enumerate(top_params_list):
    print(f"--- Final Training for Combination {i} ---")
    print("Params:", params)
    
    model = create_model(**params)
    
    final_early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_mean_absolute_percentage_error',
        patience=15,  # Stop if no improvement for 15 epochs
        restore_best_weights=True,
        verbose=1
    )
    final_lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_mean_absolute_percentage_error',
        factor=0.5,
        patience=10,
        verbose=1
    )
    final_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=f'final_model_combo_{i}.keras',
        monitor='val_mean_absolute_percentage_error',
        save_best_only=True,
        verbose=1
    )
    
    model.fit(
        train_dataset,
        validation_data=eval_dataset,
        epochs=200,
        verbose=1,
        callbacks=[final_early_stop, final_lr_scheduler, final_checkpoint]
    )
    
    train_score = model.evaluate(train_dataset, verbose=0)
    eval_score  = model.evaluate(eval_dataset, verbose=0)
    test_score  = model.evaluate(test_dataset, verbose=0)
    
    print(f"\n--- Performance for Combination {i} ---")
    print(f"Train  -> Loss: {train_score[0]:.4f}, MAE: {train_score[1]:.4f}, RMSE: {train_score[2]:.4f}, MAPE: {train_score[3]:.2f}%")
    print(f"Eval   -> Loss: {eval_score[0]:.4f}, MAE: {eval_score[1]:.4f}, RMSE: {eval_score[2]:.4f}, MAPE: {eval_score[3]:.2f}%")
    print(f"Test   -> Loss: {test_score[0]:.4f}, MAE: {test_score[1]:.4f}, RMSE: {test_score[2]:.4f}, MAPE: {test_score[3]:.2f}%\n")


--- Final Training for Combination 0 ---
Params: {'hidden_layers': 15, 'neurons': 100, 'optimizer': 'Adam', 'learning_rate': 0.0027447053581130367, 'regularization': 0.001, 'dropout_rate': 0.10666397301582889}

Epoch 1/200
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 325092192.0000 - mae: 13421.6494 - mean_absolute_percentage_error: 99.9969 - root_mean_squared_error: 18028.1914
Epoch 1: val_mean_absolute_percentage_error improved from inf to 98.11957, saving model to final_model_combo_0.keras
150/150 ━━━━━━━━━━━━━━━━━━━━ 27s 34ms/step - loss: 325081280.0000 - mae: 13421.3750 - mean_absolute_percentage_error: 99.9968 - root_mean_squared_error: 18027.9316 - val_loss: 270214432.0000 - val_mae: 12577.7656 - val_mean_absolute_percentage_error: 98.1196 - val_root_mean_squared_error: 16438.2012 - learning_rate: 0.0027
Epoch 2/200
148/150 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 322341120.0000 - mae: 13375.9219 - mean_absolute_percentage_error: 99.8399 - root_mean_squared_error: 17951.65

## Summary

1. ANNs require extensive hyperparameter tuning to achieve okay performance.
2. ANNs lack in terms of speed, precision and simplicity behind tree boosting/ensemble methods.